In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.layers import Input
from keras import backend as K
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
from tqdm import tqdm
from heamy.dataset import Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score 
from skimage import io,transform

import time

import os
import fnmatch

Using TensorFlow backend.
ERROR:tornado.application:Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7f9fe592c840>)
Traceback (most recent call last):
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/tornado/ioloop.py", line 600, in _run_callback
    ret = callback()
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 242, in enter_eventloop
    self.eventloop(self)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/eventloops.py", line 163, in loop_tk
    kernel.timer = Timer(doi)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/eventloops.py", line 151, in __init__
    self.app = Tk()
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1871, in __init__
    self.tk = _tkinter.create(screenName, baseName, className, interactive, wantobjects

In [89]:
datagen = ImageDataGenerator(
#    rotation_range=90,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=True)

def multilabelmetrics(y_true,y_pred):
    '''y_true and y_pred should be boolean np arrays
    of shape num_example x num_classes '''
    total = np.sum(y_true,axis = 0)
    tp = np.sum(y_true*y_pred,axis=0)
    tn = np.sum((1-y_true)*(1-y_pred),axis=0)
    fp = np.sum((1-y_true)*y_pred,axis=0)
    fn = np.sum(y_true*(1-y_pred),axis=0)
    return total,tp,tn,fp,fn

def combine_predictions(x,y,y1,y2,thresh,thresh1,thresh2,thresh3):
    y_pred = np.zeros((x.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh),np.uint8)
    y1_bool = np.array((y1 > thresh1),np.uint8)
    y2_bool = np.array((y2 > thresh2)*np.tile(y1[:,0]>thresh3,(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool[:,1:]
    y_pred[:,13:] = y_bool
    return y_pred

def combine_predictions_2(x,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4):#if cloud > thresh4, 
    y_pred = np.zeros((x.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh),np.uint8)
    y1_bool = np.array((y1 > thresh1),np.uint8)
    y2_bool = np.array((y2 > thresh2)*np.tile(y1[:,0]>thresh3,(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool[:,1:]
    y_pred[:,13:] = y_bool
    y_pred[((np.nonzero(y[:,1]>thresh4))[0]),:] = 0
    y_pred[((np.nonzero(y[:,1]>thresh4))[0]),14] = 1
    return y_pred

callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]

In [3]:

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

np.random.shuffle(df_train.values)

train_values = df_train.values[:36000]
val_values = df_train.values[36000:]

x_train = np.zeros((36000,200,200,3), np.float32)
x_val = np.zeros((40479-36000,200,200,3), np.float32)
y_train = []
y_val = []

i=0

for f, tags in tqdm(train_values, miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)

i=0

for f, tags in tqdm(val_values, miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_val[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_val.append(targets)
  
y_train = np.array(y_train, np.uint8)
y_val = np.array(y_val, np.uint8)


print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)


100%|██████████| 4479/4479 [00:08<00:00, 525.62it/s]

(36000, 200, 200, 3)
(36000, 17)
(4479, 200, 200, 3)
(4479, 17)


In [4]:
#subtracting mean
train_mean = np.mean(x_train,axis = 0)
x_train -= train_mean
x_val -= train_mean

In [5]:
#weather classifier (last four labels - mutually exclusive)

#y_train_w = y_train[:,-4:]
#y_val_w = y_val[:,-4:]
# x_val = np.array([])
# y_val_w = np.array([])
# dataset = Dataset(X_train=x_train, y_train=y_train_w,X_test=None, y_test=None)
# dataset.split(test_size=0.1,inplace=True)

#print(x_train.shape)
# print(y_train_w.shape)
# print(x_val.shape)
# print(y_val_w.shape)

In [13]:
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(17, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [14]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])    
model.fit_generator(datagen.flow(x_train,y_train, batch_size = 128), validation_data=(x_val, y_val),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
282/281 [==============================] - 239s - loss: 0.1993 - acc: 0.9239 - val_loss: 0.1705 - val_acc: 0.9388
Epoch 2/10
282/281 [==============================] - 236s - loss: 0.1668 - acc: 0.9359 - val_loss: 0.1540 - val_acc: 0.9420
Epoch 3/10
282/281 [==============================] - 236s - loss: 0.1615 - acc: 0.9379 - val_loss: 0.1491 - val_acc: 0.9447
Epoch 4/10
282/281 [==============================] - 236s - loss: 0.1595 - acc: 0.9385 - val_loss: 0.1452 - val_acc: 0.9454
Epoch 5/10
282/281 [==============================] - 236s - loss: 0.1574 - acc: 0.9395 - val_loss: 0.1434 - val_acc: 0.9459
Epoch 6/10
282/281 [==============================] - 237s - loss: 0.1561 - acc: 0.9397 - val_loss: 0.1427 - val_acc: 0.9458
Epoch 7/10
282/281 [==============================] - 237s - loss: 0.1558 - acc: 0.9401 - val_loss: 0.1417 - val_acc: 0.9465
Epoch 8/10
282/281 [==============================] - 237s - loss: 0.1541 - acc: 0.9407 - val_loss: 0.1411 - val_acc: 0.9468


In [15]:
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.81119586456
thresh: 0.1 	F2 score: 0.848830268377
thresh: 0.15 	F2 score: 0.86416940206
thresh: 0.2 	F2 score: 0.872163694737
thresh: 0.25 	F2 score: 0.873875452547
thresh: 0.3 	F2 score: 0.870081199074
thresh: 0.35 	F2 score: 0.86535669398


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [16]:
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy'])    
model.fit_generator(datagen.flow(x_train,y_train, batch_size = 128), validation_data=(x_val, y_val),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
282/281 [==============================] - 240s - loss: 0.1517 - acc: 0.9414 - val_loss: 0.1425 - val_acc: 0.9477
Epoch 2/10
282/281 [==============================] - 237s - loss: 0.1506 - acc: 0.9418 - val_loss: 0.1413 - val_acc: 0.9483
Epoch 3/10
282/281 [==============================] - 237s - loss: 0.1506 - acc: 0.9416 - val_loss: 0.1386 - val_acc: 0.9484
Epoch 4/10
282/281 [==============================] - 237s - loss: 0.1492 - acc: 0.9423 - val_loss: 0.1390 - val_acc: 0.9480
Epoch 5/10
282/281 [==============================] - 237s - loss: 0.1492 - acc: 0.9427 - val_loss: 0.1388 - val_acc: 0.9489
Epoch 6/10
282/281 [==============================] - 237s - loss: 0.1492 - acc: 0.9424 - val_loss: 0.1372 - val_acc: 0.9485
Epoch 7/10
282/281 [==============================] - 237s - loss: 0.1483 - acc: 0.9428 - val_loss: 0.1369 - val_acc: 0.9483
Epoch 8/10
282/281 [==============================] - 236s - loss: 0.1481 - acc: 0.9428 - val_loss: 0.1381 - val_acc: 0.9490


In [17]:
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.803230009448
thresh: 0.1 	F2 score: 0.844400364029
thresh: 0.15 	F2 score: 0.863639557577
thresh: 0.2 	F2 score: 0.871966723545
thresh: 0.25 	F2 score: 0.87566910195
thresh: 0.3 	F2 score: 0.872494416247
thresh: 0.35 	F2 score: 0.869288732875


In [18]:
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy'])    
model.fit_generator(datagen.flow(x_train,y_train, batch_size = 128), validation_data=(x_val, y_val),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
282/281 [==============================] - 238s - loss: 0.1469 - acc: 0.9434 - val_loss: 0.1369 - val_acc: 0.9493
Epoch 2/10
282/281 [==============================] - 236s - loss: 0.1464 - acc: 0.9437 - val_loss: 0.1367 - val_acc: 0.9492
Epoch 3/10
282/281 [==============================] - 236s - loss: 0.1445 - acc: 0.9442 - val_loss: 0.1360 - val_acc: 0.9493
Epoch 4/10
282/281 [==============================] - 236s - loss: 0.1461 - acc: 0.9435 - val_loss: 0.1371 - val_acc: 0.9489
Epoch 5/10
282/281 [==============================] - 236s - loss: 0.1460 - acc: 0.9437 - val_loss: 0.1367 - val_acc: 0.9493
Epoch 6/10
282/281 [==============================] - 237s - loss: 0.1460 - acc: 0.9437 - val_loss: 0.1372 - val_acc: 0.9491


In [19]:
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.801496808014
thresh: 0.1 	F2 score: 0.843709131221
thresh: 0.15 	F2 score: 0.862725849959
thresh: 0.2 	F2 score: 0.873129729499
thresh: 0.25 	F2 score: 0.875997212448
thresh: 0.3 	F2 score: 0.8764597919
thresh: 0.35 	F2 score: 0.871721456544


In [20]:
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])    
model.fit_generator(datagen.flow(x_train,y_train, batch_size = 128), validation_data=(x_val, y_val),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
282/281 [==============================] - 239s - loss: 0.1453 - acc: 0.9436 - val_loss: 0.1369 - val_acc: 0.9495
Epoch 2/10
282/281 [==============================] - 236s - loss: 0.1448 - acc: 0.9443 - val_loss: 0.1364 - val_acc: 0.9496
Epoch 3/10
282/281 [==============================] - 236s - loss: 0.1453 - acc: 0.9440 - val_loss: 0.1369 - val_acc: 0.9494
Epoch 4/10
282/281 [==============================] - 236s - loss: 0.1447 - acc: 0.9442 - val_loss: 0.1367 - val_acc: 0.9494
Epoch 5/10
282/281 [==============================] - 236s - loss: 0.1453 - acc: 0.9439 - val_loss: 0.1369 - val_acc: 0.9497


In [21]:
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.802104357584
thresh: 0.1 	F2 score: 0.844076097056
thresh: 0.15 	F2 score: 0.863527317305
thresh: 0.2 	F2 score: 0.87385096256
thresh: 0.25 	F2 score: 0.879230965904
thresh: 0.3 	F2 score: 0.877471758944
thresh: 0.35 	F2 score: 0.873553301541


In [22]:
def optimise_f2_thresholds(x_init, y, p, num_classes=17, verbose=True, resolution=100):
  def mf(x):
    p2 = np.zeros_like(p)
    for i in range(num_classes):
      p2[:, i] = (p[:, i] > x[i]).astype(np.int)
    score = fbeta_score(y, p2, beta=2, average='samples')
    return score

  x = list(x_init)
  for i in range(num_classes):
    best_i2 = 0
    best_score = 0
    for i2 in range(resolution):
      i2 /= resolution
      x[i] = i2
      score = mf(x)
      if score > best_score:
        best_i2 = i2
        best_score = score
    x[i] = best_i2
    if verbose:
      print(i, best_i2, best_score)

  return x

thresh = optimise_f2_thresholds([0.2]*17,y_val,y_pred)

0 0.18 0.87385096256
1 0.22 0.873965841508
2 0.26 0.874019578792
3 0.16 0.874079495165
4 0.35 0.874171042403
5 0.28 0.874308619631
6 0.23 0.874327042615
7 0.31 0.87600803652
8 0.28 0.877150794416
9 0.26 0.877752257782
10 0.3 0.878754952535
11 0.23 0.879157448546
12 0.19 0.879229999994
13 0.27 0.879984148763
14 0.17 0.880169663533
15 0.24 0.88044378153
16 0.27 0.881502188608


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [23]:
thresh = optimise_f2_thresholds(thresh,y_val,y_pred)

0 0.18 0.881502188608
1 0.22 0.881502188608
2 0.26 0.881502188608
3 0.16 0.881502188608
4 0.35 0.881502188608
5 0.28 0.881502188608
6 0.23 0.881502188608
7 0.31 0.881502188608
8 0.28 0.881502188608
9 0.26 0.881502188608
10 0.3 0.881502188608
11 0.23 0.881502188608
12 0.19 0.881502188608
13 0.27 0.881502188608
14 0.17 0.881502188608
15 0.24 0.881502188608
16 0.27 0.881502188608


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [24]:
total,tp,tn,fp,fn = multilabelmetrics(y_val,np.array(y_pred>np.array(thresh)))
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels)

,FN,FP,TN,TP,Total
blow_down,13,0,4465,1,14
bare_ground,72,112,4260,35,107
conventional_mine,10,6,4458,5,15
blooming,34,15,4425,5,39
artisinal_mine,11,19,4428,21,32
selective_logging,33,13,4425,8,41
slash_burn,26,3,4448,2,28
cultivation,232,334,3678,235,467
habitation,115,380,3720,264,379
road,116,589,3039,735,851


In [25]:
model.save_weights('inc_weights')

In [ ]:
for layer in base_model.layers:
    layer.trainable = True
#continue with reduced learning rate
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
callbacks = [#EarlyStopping(monitor='val_loss', patience=2, verbose=0),
ModelCheckpoint('inc_weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', 
                verbose=0, save_best_only=True,)]

model.fit_generator(datagen.flow(x_train,y_train, batch_size = 64), validation_data=(x_val, y_val),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/10
563/562 [==============================] - 719s - loss: 0.1174 - acc: 0.9556 - val_loss: 0.1018 - val_acc: 0.9616
Epoch 2/10
563/562 [==============================] - 705s - loss: 0.1012 - acc: 0.9612 - val_loss: 0.0986 - val_acc: 0.9631
Epoch 3/10
563/562 [==============================] - 705s - loss: 0.0945 - acc: 0.9636 - val_loss: 0.0933 - val_acc: 0.9642
Epoch 4/10
563/562 [==============================] - 703s - loss: 0.0896 - acc: 0.9655 - val_loss: 0.0935 - val_acc: 0.9643
Epoch 5/10
563/562 [==============================] - 705s - loss: 0.0860 - acc: 0.9667 - val_loss: 0.0918 - val_acc: 0.9651
Epoch 6/10
563/562 [==============================] - 704s - loss: 0.0823 - acc: 0.9682 - val_loss: 0.0936 - val_acc: 0.9649
Epoch 7/10
563/562 [==============================] - 704s - loss: 0.0796 - acc: 0.9694 - val_loss: 0.0926 - val_acc: 0.9651
Epoch 8/10
563/562 [==============================] - 706s - loss: 0.0756 - acc: 0.9708 - val_loss: 0.0923 - val_acc: 0.9658


In [28]:
model.fit_generator(datagen.flow(x_train,y_train, batch_size = 64), validation_data=(x_val, y_val),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,initial_epoch=9,
                  )

Epoch 10/10
563/562 [==============================] - 703s - loss: 0.0697 - acc: 0.9729 - val_loss: 0.0957 - val_acc: 0.9642


In [30]:
model.load_weights('inc_weights.04-0.09.hdf5')
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.897050859774
thresh: 0.1 	F2 score: 0.913967605522
thresh: 0.15 	F2 score: 0.91961846729
thresh: 0.2 	F2 score: 0.921813323781
thresh: 0.25 	F2 score: 0.921611665731
thresh: 0.3 	F2 score: 0.919595515965
thresh: 0.35 	F2 score: 0.915540603322


In [31]:
thresh_1_val = optimise_f2_thresholds([0.2]*17,y_val,y_pred)

0 0.05 0.921933577047
1 0.13 0.922095760924
2 0.07 0.922133862111
3 0.4 0.922381212767
4 0.16 0.922446459528
5 0.33 0.922645440039
6 0.19 0.922645440039
7 0.27 0.922863873856
8 0.22 0.922979190859
9 0.23 0.923069268089
10 0.19 0.923129157551
11 0.26 0.923300278924
12 0.1 0.923708021007
13 0.15 0.923766491402
14 0.16 0.92394439653
15 0.23 0.924044823182
16 0.31 0.924262329554


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [32]:
thresh_1_val = optimise_f2_thresholds(thresh_1_val,y_val,y_pred)

0 0.05 0.924262329554
1 0.13 0.924262329554
2 0.07 0.924262329554
3 0.4 0.924262329554
4 0.16 0.924262329554
5 0.33 0.924262329554
6 0.19 0.924262329554
7 0.27 0.924262329554
8 0.22 0.924262329554
9 0.23 0.924262329554
10 0.19 0.924262329554
11 0.26 0.924262329554
12 0.1 0.924262329554
13 0.25 0.924267387957
14 0.16 0.924267387957
15 0.23 0.924267387957
16 0.31 0.924267387957


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [33]:
total,tp,tn,fp,fn = multilabelmetrics(y_val,np.array(y_pred>np.array(thresh_1_val)))
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels)

,FN,FP,TN,TP,Total
blow_down,9,6,4459,5,14
bare_ground,65,91,4281,42,107
conventional_mine,7,4,4460,8,15
blooming,31,3,4437,8,39
artisinal_mine,0,17,4430,32,32
selective_logging,22,26,4412,19,41
slash_burn,26,0,4451,2,28
cultivation,132,318,3694,335,467
habitation,82,183,3917,297,379
road,84,324,3304,767,851


In [34]:
model.save_weights("inc_weights_1")

In [36]:
model.load_weights("inc_weights")

for layer in base_model.layers:
    layer.trainable = True

datagen1 = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)

    
#continue with reduced learning rate
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
callbacks = [#EarlyStopping(monitor='val_loss', patience=2, verbose=0),
ModelCheckpoint('inc_weights_2.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', 
                verbose=0, save_best_only=True,)]

model.fit_generator(datagen1.flow(x_train,y_train, batch_size = 64), validation_data=(x_val, y_val),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,
                  )

Epoch 1/10
563/562 [==============================] - 719s - loss: 0.1282 - acc: 0.9525 - val_loss: 0.1079 - val_acc: 0.9591
Epoch 2/10
563/562 [==============================] - 704s - loss: 0.1128 - acc: 0.9576 - val_loss: 0.0993 - val_acc: 0.9622
Epoch 3/10
563/562 [==============================] - 705s - loss: 0.1075 - acc: 0.9598 - val_loss: 0.0962 - val_acc: 0.9625
Epoch 4/10
563/562 [==============================] - 703s - loss: 0.1042 - acc: 0.9610 - val_loss: 0.0966 - val_acc: 0.9620
Epoch 5/10
563/562 [==============================] - 704s - loss: 0.1019 - acc: 0.9619 - val_loss: 0.0942 - val_acc: 0.9641
Epoch 6/10
563/562 [==============================] - 703s - loss: 0.1001 - acc: 0.9624 - val_loss: 0.0927 - val_acc: 0.9645
Epoch 7/10
563/562 [==============================] - 695s - loss: 0.0997 - acc: 0.9625 - val_loss: 0.0945 - val_acc: 0.9636
Epoch 8/10
563/562 [==============================] - 699s - loss: 0.0977 - acc: 0.9633 - val_loss: 0.0921 - val_acc: 0.9646


In [37]:
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.883296117599
thresh: 0.1 	F2 score: 0.905590204959
thresh: 0.15 	F2 score: 0.915810485909
thresh: 0.2 	F2 score: 0.920296971336
thresh: 0.25 	F2 score: 0.920828006783
thresh: 0.3 	F2 score: 0.920129832671
thresh: 0.35 	F2 score: 0.914566275125


In [38]:
model.fit_generator(datagen1.flow(x_train,y_train, batch_size = 64), validation_data=(x_val, y_val),
                  epochs=15, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,initial_epoch=10
                  )

Epoch 11/15
563/562 [==============================] - 699s - loss: 0.0941 - acc: 0.9644 - val_loss: 0.0908 - val_acc: 0.9659
Epoch 12/15
563/562 [==============================] - 700s - loss: 0.0938 - acc: 0.9646 - val_loss: 0.0891 - val_acc: 0.9668
Epoch 13/15
563/562 [==============================] - 698s - loss: 0.0923 - acc: 0.9651 - val_loss: 0.0921 - val_acc: 0.9650
Epoch 14/15
563/562 [==============================] - 699s - loss: 0.0909 - acc: 0.9656 - val_loss: 0.0894 - val_acc: 0.9655
Epoch 15/15
563/562 [==============================] - 699s - loss: 0.0896 - acc: 0.9660 - val_loss: 0.0896 - val_acc: 0.9651


In [39]:
y_pred = model.predict(x_val,batch_size=16)


NameError: name 'y_val_w' is not defined

In [40]:
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.895501299997
thresh: 0.1 	F2 score: 0.913870185048
thresh: 0.15 	F2 score: 0.920357930064
thresh: 0.2 	F2 score: 0.92292332587
thresh: 0.25 	F2 score: 0.922960735659
thresh: 0.3 	F2 score: 0.921214164823
thresh: 0.35 	F2 score: 0.91726728678


In [41]:
model.fit_generator(datagen1.flow(x_train,y_train, batch_size = 64), validation_data=(x_val, y_val),
                  epochs=20, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,initial_epoch=15
                  )

Epoch 16/20
563/562 [==============================] - 699s - loss: 0.0882 - acc: 0.9666 - val_loss: 0.0886 - val_acc: 0.9664
Epoch 17/20
563/562 [==============================] - 698s - loss: 0.0883 - acc: 0.9665 - val_loss: 0.0909 - val_acc: 0.9655
Epoch 18/20
563/562 [==============================] - 699s - loss: 0.0865 - acc: 0.9670 - val_loss: 0.0924 - val_acc: 0.9649
Epoch 19/20
563/562 [==============================] - 702s - loss: 0.0849 - acc: 0.9676 - val_loss: 0.0900 - val_acc: 0.9655
Epoch 20/20
563/562 [==============================] - 704s - loss: 0.0841 - acc: 0.9682 - val_loss: 0.0932 - val_acc: 0.9653


In [42]:
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.901041947935
thresh: 0.1 	F2 score: 0.913054439367
thresh: 0.15 	F2 score: 0.917572305411
thresh: 0.2 	F2 score: 0.919269592649
thresh: 0.25 	F2 score: 0.918651024012
thresh: 0.3 	F2 score: 0.917730770771
thresh: 0.35 	F2 score: 0.915255506917


In [57]:
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(17, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = True
    
model.load_weights('inc_weights_2.15-0.09.hdf5')

In [58]:
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.898413025855
thresh: 0.1 	F2 score: 0.914924595428
thresh: 0.15 	F2 score: 0.920018977986
thresh: 0.2 	F2 score: 0.92172635772
thresh: 0.25 	F2 score: 0.921200496649
thresh: 0.3 	F2 score: 0.918887740825
thresh: 0.35 	F2 score: 0.917874984399


In [60]:
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.00005),
              metrics=['accuracy'])

model.fit_generator(datagen1.flow(x_train,y_train, batch_size = 64), validation_data=(x_val, y_val),
                  epochs=21, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,initial_epoch=20
                  )

Epoch 21/21
563/562 [==============================] - 710s - loss: 0.0837 - acc: 0.9681 - val_loss: 0.0890 - val_acc: 0.9656


In [61]:
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.901033220695
thresh: 0.1 	F2 score: 0.915732459627
thresh: 0.15 	F2 score: 0.921695326799
thresh: 0.2 	F2 score: 0.92353546239
thresh: 0.25 	F2 score: 0.922672385039
thresh: 0.3 	F2 score: 0.920683995537
thresh: 0.35 	F2 score: 0.916658735851


In [62]:
model.save_weights('inc_weights_2.20-0.09.hdf5')
model.fit_generator(datagen1.flow(x_train,y_train, batch_size = 64), validation_data=(x_val, y_val),
                  epochs=25, steps_per_epoch=x_train.shape[0]/ 64, callbacks=callbacks,initial_epoch=21
                  )

Epoch 22/25
563/562 [==============================] - 704s - loss: 0.0821 - acc: 0.9689 - val_loss: 0.0905 - val_acc: 0.9654
Epoch 23/25
563/562 [==============================] - 702s - loss: 0.0801 - acc: 0.9695 - val_loss: 0.0899 - val_acc: 0.9662
Epoch 24/25
563/562 [==============================] - 701s - loss: 0.0787 - acc: 0.9699 - val_loss: 0.0911 - val_acc: 0.9659
Epoch 25/25
563/562 [==============================] - 699s - loss: 0.0779 - acc: 0.9700 - val_loss: 0.0924 - val_acc: 0.9653


In [63]:
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.903340179116
thresh: 0.1 	F2 score: 0.915208858888
thresh: 0.15 	F2 score: 0.920071334831
thresh: 0.2 	F2 score: 0.920699965052
thresh: 0.25 	F2 score: 0.919560859574
thresh: 0.3 	F2 score: 0.916977289297
thresh: 0.35 	F2 score: 0.915211930744


In [64]:
model.load_weights('inc_weights_2.20-0.09.hdf5')
y_pred = model.predict(x_val,batch_size=16)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.901033220695
thresh: 0.1 	F2 score: 0.915732459627
thresh: 0.15 	F2 score: 0.921695326799
thresh: 0.2 	F2 score: 0.92353546239
thresh: 0.25 	F2 score: 0.922672385039
thresh: 0.3 	F2 score: 0.920683995537
thresh: 0.35 	F2 score: 0.916658735851


In [65]:
thresh_2_val = optimise_f2_thresholds([0.2]*17,y_val,y_pred)

0 0.23 0.923549416398
1 0.32 0.923910625328
2 0.57 0.923977767663
3 0.36 0.924250140772
4 0.49 0.924269671023
5 0.25 0.924313156
6 0.16 0.924358340405
7 0.32 0.924744496042
8 0.21 0.924828213775
9 0.17 0.92493470304
10 0.19 0.924996207985
11 0.18 0.925107626486
12 0.15 0.925306856072
13 0.33 0.925599478955
14 0.04 0.926986167868
15 0.2 0.926986167868
16 0.16 0.927050533219


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [66]:
thresh_2_val = optimise_f2_thresholds(thresh_2_val,y_val,y_pred)

0 0.23 0.927050533219
1 0.32 0.927050533219
2 0.57 0.927050533219
3 0.36 0.927050533219
4 0.49 0.927050533219
5 0.25 0.927050533219
6 0.16 0.927050533219
7 0.32 0.927050533219
8 0.21 0.927050533219
9 0.17 0.927050533219
10 0.19 0.927050533219
11 0.16 0.927056085924
12 0.15 0.927056085924
13 0.33 0.927056085924
14 0.04 0.927056085924
15 0.2 0.927056085924
16 0.16 0.927056085924


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [67]:
total,tp,tn,fp,fn = multilabelmetrics(y_val,np.array(y_pred>np.array(thresh_2_val)))
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels)

,FN,FP,TN,TP,Total
blow_down,10,6,4459,4,14
bare_ground,52,94,4278,55,107
conventional_mine,2,7,4457,13,15
blooming,25,16,4424,14,39
artisinal_mine,4,6,4441,28,32
selective_logging,22,34,4404,19,41
slash_burn,22,16,4435,6,28
cultivation,154,255,3757,313,467
habitation,59,223,3877,320,379
road,65,322,3306,786,851


In [68]:
model.save_weights('inc_weights_2')
np.save('inc_train_mean',train_mean)

In [70]:
model.save('inc_model')

In [71]:
K.clear_session()

In [76]:
np.save('thresh_2_val',thresh_2_val)
np.save('thresh_1_val',thresh_1_val)

In [77]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [80]:
#F2 score on training set


x_val = []
x_train = []
x_train = np.zeros((40479,200,200,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)

y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [07:51<00:00, 85.92it/s]

(40479, 200, 200, 3)
(40479, 17)


In [81]:
train_mean = np.load('inc_train_mean.npy')
thresh_1_val = np.load('thresh_1_val.npy')
thresh_2_val = np.load('thresh_2_val.npy')
x_train -= train_mean

In [82]:
from keras.models import load_model
model = load_model('inc_model')
model.load_weights('inc_weights_1')

In [83]:
y_pred = model.predict(x_train,batch_size=128)

In [84]:
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_train, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.908883556969
thresh: 0.1 	F2 score: 0.926870720572
thresh: 0.15 	F2 score: 0.932795624168
thresh: 0.2 	F2 score: 0.934768147342
thresh: 0.25 	F2 score: 0.934201197186
thresh: 0.3 	F2 score: 0.931977226576
thresh: 0.35 	F2 score: 0.928789929143


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [85]:
def optimise_f2_thresholds(x_init, y, p, num_classes=17, verbose=True, resolution=100):
  def mf(x):
    p2 = np.zeros_like(p)
    for i in range(num_classes):
      p2[:, i] = (p[:, i] > x[i]).astype(np.int)
    score = fbeta_score(y, p2, beta=2, average='samples')
    return score

  x = list(x_init)
  for i in range(num_classes):
    best_i2 = 0
    best_score = 0
    for i2 in range(resolution):
      i2 /= resolution
      x[i] = i2
      score = mf(x)
      if score > best_score:
        best_i2 = i2
        best_score = score
    x[i] = best_i2
    if verbose:
      print(i, best_i2, best_score)

  return x

In [86]:
thresh_1_train = optimise_f2_thresholds([0.2]*17,y_train,y_pred)

0 0.08 0.934796809864
1 0.13 0.934924463979
2 0.06 0.934946239251
3 0.23 0.934994882101
4 0.25 0.935006227422
5 0.33 0.935151719074
6 0.11 0.935197988199
7 0.27 0.935400739995
8 0.25 0.935424750667
9 0.24 0.935548874133
10 0.2 0.935548874133
11 0.23 0.93558877266
12 0.14 0.935792282161
13 0.11 0.935850650531
14 0.17 0.935901812568
15 0.23 0.936031475652
16 0.18 0.936067226427


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [87]:
thresh_1_train = optimise_f2_thresholds(thresh_1_train,y_train,y_pred)

0 0.08 0.936067226427
1 0.13 0.936067226427
2 0.06 0.936067226427
3 0.23 0.936067226427
4 0.25 0.936067226427
5 0.33 0.936067226427
6 0.11 0.936067226427
7 0.27 0.936067226427
8 0.25 0.936067226427
9 0.24 0.936067226427
10 0.2 0.936067226427
11 0.23 0.936067226427
12 0.14 0.936067226427
13 0.11 0.936067226427
14 0.17 0.936067226427
15 0.23 0.936067226427
16 0.18 0.936067226427


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [90]:
total,tp,tn,fp,fn = multilabelmetrics(y_train,np.array(y_pred>np.array(thresh_1_train)))
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels)

,FN,FP,TN,TP,Total
blow_down,50,76,40305,48,98
bare_ground,430,711,38906,432,862
conventional_mine,32,51,40328,68,100
blooming,231,171,39976,101,332
artisinal_mine,29,97,40043,310,339
selective_logging,150,219,39920,190,340
slash_burn,150,146,40124,59,209
cultivation,1213,2786,33216,3264,4477
habitation,679,1331,35488,2981,3660
road,671,2319,30089,7400,8071


In [91]:
np.save('thresh_1_train',thresh_1_train)

In [92]:
model.load_weights('inc_weights_2')

In [93]:
y_pred = model.predict(x_train,batch_size=128)

In [94]:
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_train, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.913437259177
thresh: 0.1 	F2 score: 0.930891371088
thresh: 0.15 	F2 score: 0.93769219526
thresh: 0.2 	F2 score: 0.940144972476
thresh: 0.25 	F2 score: 0.940142050986
thresh: 0.3 	F2 score: 0.938630106111
thresh: 0.35 	F2 score: 0.935727927362


In [95]:
thresh_2_train = optimise_f2_thresholds([0.2]*17,y_train,y_pred)

0 0.18 0.940150692316
1 0.29 0.940345332275
2 0.47 0.940357265931
3 0.34 0.940514574253
4 0.18 0.940515328251
5 0.4 0.940630459791
6 0.18 0.940656469208
7 0.25 0.940987635956
8 0.28 0.941044528442
9 0.2 0.941044528442
10 0.24 0.941063298677
11 0.2 0.941063298677
12 0.22 0.941072012511
13 0.31 0.941245029821
14 0.12 0.941501059424
15 0.15 0.941571475599
16 0.26 0.941656985786


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [96]:
thresh_2_train = optimise_f2_thresholds(thresh_2_train,y_train,y_pred)

0 0.18 0.941656985786
1 0.29 0.941656985786
2 0.47 0.941656985786
3 0.34 0.941656985786
4 0.18 0.941656985786
5 0.4 0.941656985786
6 0.18 0.941656985786
7 0.25 0.941656985786
8 0.28 0.941656985786
9 0.2 0.941656985786
10 0.24 0.941656985786
11 0.2 0.941656985786
12 0.22 0.941656985786
13 0.31 0.941656985786
14 0.12 0.941656985786
15 0.15 0.941656985786
16 0.26 0.941656985786


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [97]:
total,tp,tn,fp,fn = multilabelmetrics(y_train,np.array(y_pred>np.array(thresh_2_train)))
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels)

,FN,FP,TN,TP,Total
blow_down,36,63,40318,62,98
bare_ground,335,716,38901,527,862
conventional_mine,20,28,40351,80,100
blooming,214,149,39998,118,332
artisinal_mine,16,81,40059,323,339
selective_logging,155,147,39992,185,340
slash_burn,121,124,40146,88,209
cultivation,958,3235,32767,3519,4477
habitation,519,1389,35430,3141,3660
road,567,2062,30346,7504,8071


In [98]:
np.save('thresh_2_train',thresh_2_train)

In [99]:
print(thresh_1_train)
print(thresh_2_train)
print(thresh_1_val)
print(thresh_2_val)

[0.08, 0.13, 0.06, 0.23, 0.25, 0.33, 0.11, 0.27, 0.25, 0.24, 0.2, 0.23, 0.14, 0.11, 0.17, 0.23, 0.18]
[0.18, 0.29, 0.47, 0.34, 0.18, 0.4, 0.18, 0.25, 0.28, 0.2, 0.24, 0.2, 0.22, 0.31, 0.12, 0.15, 0.26]
[ 0.05  0.13  0.07  0.4   0.16  0.33  0.19  0.27  0.22  0.23  0.19  0.26
  0.1   0.25  0.16  0.23  0.31]
[ 0.23  0.32  0.57  0.36  0.49  0.25  0.16  0.32  0.21  0.17  0.19  0.16
  0.15  0.33  0.04  0.2   0.16]


In [107]:
K.clear_session()

In [2]:
thresh_1_train = np.load('thresh_1_train.npy')
thresh_2_train = np.load('thresh_2_train.npy')
thresh_1_val = np.load('thresh_1_val.npy')
thresh_2_val = np.load('thresh_2_val.npy')
train_mean = np.load('inc_train_mean.npy')

base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(17, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [3]:
#Test set
x_train = []
x_val = []
x_test = np.zeros((61191,200,200,3), np.float32)
y_train = []

df_test = pd.read_csv('sample_submission_v2.csv')

i = 0 
for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('test-jpg/{}.jpg'.format(f))
    x_test[i,:,:,:] = np.array(cv2.resize(img, (200, 200)),np.float32)/255.#139 minimum size for inception
    i+=1
print(x_test.shape)

100%|██████████| 61191/61191 [08:48<00:00, 115.68it/s]

(61191, 200, 200, 3)


In [4]:
x_test -= train_mean
model.load_weights('inc_weights_2')
y_test = model.predict(x_test,batch_size=64)
y_pred = np.array(y_test>np.array(thresh_2_val))

In [6]:
labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

In [7]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_inc_1.csv', index=False)
#test set score:0.92398

In [8]:
y_pred = np.array(y_test>np.array(thresh_2_train))

In [9]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_inc_2.csv', index=False)
#test set score:0.92391

In [10]:
model.load_weights('inc_weights_1')
y_test = model.predict(x_test,batch_size=64)
y_pred = np.array(y_test>np.array(thresh_2_val))

In [11]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_inc_3.csv', index=False)
#test set score:0.91841

In [12]:
y_pred = np.array(y_test>np.array(thresh_2_train))

In [13]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_inc_4.csv', index=False)
#test set score: